In [4]:
import sys
import requests
from requests import request
import concurrent.futures
import os
import pdb
url = 'https://kf-api-dataservice.kidsfirstdrc.org/'
requests.packages.urllib3.disable_warnings()

In [5]:
def query_ds(line, url):
    line = line.rstrip('\n')
    info = line.split('\t')
    bs_id = info[1]
    task_id = info[2]
    gf_url = url + 'genomic-files?biospecimen_id=' + bs_id
    gf_related = request('GET', gf_url, verify=False)
    i = 0
    n = 6
    cram_fn = task_id + '.cram'
    gvcf_fn = task_id + '.g.vcf.gz'
    crai_fn = cram_fn + '.crai'
    tbi_fn = gvcf_fn + '.tbi'
    vcf_suffix = 'CGP.filtered.deNovo.vep.vcf.gz'
    vcf_tbi_suffix = vcf_suffix + '.tbi'
    vcf_fn = ''
    vcf_tbi_fn = ''
    key_list = [cram_fn, crai_fn, gvcf_fn, tbi_fn]
    fn_dict = {cram_fn: 0, crai_fn: 0, gvcf_fn: 0, tbi_fn: 0}
    check = []
    # pdb.set_trace()
    for result in gf_related.json()['results']:
        if i == n:
            break
        gf_id = result['_links']['self'].split('/')[2]
        fname = result['file_name']
        check.append(fname + str(len(fname)))
        if fname in fn_dict:
            i+=1
            fn_dict[fname] = gf_id
        elif len(fname) > 30 and fname[-30:] == vcf_suffix:
            i += 1
            vcf_fn = fname
            fn_dict[fname] = gf_id
        elif len(fname) > 34 and fname[-34:] == vcf_tbi_suffix:
            i += 1
            vcf_tbi_fn = fname
            fn_dict[fname] = gf_id
    key_list.append(vcf_fn)
    key_list.append(vcf_tbi_fn)
    for key in key_list:
        try:
            line += '\t' + key + '\t' + fn_dict[key]
        except:
            sys.stderr.write('Could not find ' + key + ' for ' + bs_id + '\nassociated files:' + '\n'.join(check) + '\n')
            line += '\tNA\tNA'
    return line + '\n'
    
    

In [6]:
tbl = open('/Users/brownm28/Documents/2019-Jan-30_reharmonization/gf_id_inventory/washu_tbl.txt')
head = next(tbl)
head = head.rstrip('\n') + '\tcram fname\tcram gf_id\tcrai fname\tcrai gf_id\tgvcf fname\tgvcf gf_id\tgvcf tbi fname\tgvcftbi gf_id\tjoint vcf tbi fname\tjoint vcftbi gf_id\n'
out = open('/Users/brownm28/Documents/2019-Jan-30_reharmonization/gf_id_inventory/washu_tbl_w_gf_ids.txt', 'w')
out.write(head)
x = 1
m = 50
th = 24
with concurrent.futures.ThreadPoolExecutor(th) as bs_exec:
    bs_results = {
    bs_exec.submit(query_ds, line, url): line for line in tbl}
    for bs_result in concurrent.futures.as_completed(bs_results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' bs ids\n')
            sys.stderr.flush()
        update = bs_result.result()
        try:
            out.write(update)
        except:
            pdb.set_trace()
        x += 1
out.close()
sys.stderr.write('Done!')

Could not find  for BS_2VDPJ2V4
associated files:0368d899-aab8-49e7-8aab-477e215acea1.cram41
281f623abcf445be8f5035c7fd877b91.bam36
0368d899-aab8-49e7-8aab-477e215acea1.cram.crai46
0368d899-aab8-49e7-8aab-477e215acea1.g.vcf.gz45
0368d899-aab8-49e7-8aab-477e215acea1.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_2VDPJ2V4
associated files:0368d899-aab8-49e7-8aab-477e215acea1.cram41
281f623abcf445be8f5035c7fd877b91.bam36
0368d899-aab8-49e7-8aab-477e215acea1.cram.crai46
0368d899-aab8-49e7-8aab-477e215acea1.g.vcf.gz45
0368d899-aab8-49e7-8aab-477e215acea1.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_PEBS6CTY
associated files:05036423-5311-46a2-aed2-f4a183941e9b.cram41
7abbd09c62c24de1a098c0e01e316db3.bam36
05036423-5311-46a2-aed2-f4a183941e9b.cram.crai46
05036423-5311-46a2-aed2-f4a183941e9b.g.vcf.gz45
05036423-5311-46a

Could not find  for BS_0Y10ZP0C
associated files:0fbc6020-02cf-46ff-b472-7b3800008444.cram41
c52e770966044386b6715709ced48885.bam36
0fbc6020-02cf-46ff-b472-7b3800008444.cram.crai46
0fbc6020-02cf-46ff-b472-7b3800008444.g.vcf.gz45
0fbc6020-02cf-46ff-b472-7b3800008444.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
fcc6436b-5874-43d8-98a7-973c421ea96c.vcf.gz43
d00e3fd2-3a54-41dd-a130-356faecc3766.CGP.filtered.deNovo.vep.vcf.gz.tbi71
fcc6436b-5874-43d8-98a7-973c421ea96c.vcf.gz.tbi47
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_FZWMKK3G
associated files:0cfc8ea0-a322-47cc-b652-603a04ba2352.cram41
dc14ae2b793f4bb98d6bba6dad36993e.bam36
0cfc8ea0-a322-47cc-b652-603a04ba2352.cram.crai46
0cfc8ea0-a322-47cc-b652-603a04ba2352.g.vcf.gz45
0cfc8ea0-a322-47cc-b652-603a04ba2352.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
dbd65ad2-4809-4f03-bd9b-2406941156bc.vcf.gz43
dbd65ad2-4809-4f03-bd9b-2406941156bc.vcf.gz.tbi47
9f157377-ee64-4608-bc63-d66772

Could not find  for BS_E5XBSHFN
associated files:1a8e9532-105d-4c46-bf31-4e2d747647d6.cram41
95a82b600eaf455e9a7a9a769ae6e748.bam36
1a8e9532-105d-4c46-bf31-4e2d747647d6.cram.crai46
1a8e9532-105d-4c46-bf31-4e2d747647d6.g.vcf.gz45
1a8e9532-105d-4c46-bf31-4e2d747647d6.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
87c007d4-d91f-48ef-8e47-2604b3d3102b.vcf.gz.tbi47
e2a5c362-0179-4fda-a387-6934b38f43b4.CGP.filtered.deNovo.vep.vcf.gz67
87c007d4-d91f-48ef-8e47-2604b3d3102b.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_K6D0448D
associated files:1a5e47cb-6b89-44d4-821e-6e1f5e5438f4.cram41
c81cc28d5bf14159a2c995762c1865c5.bam36
1a5e47cb-6b89-44d4-821e-6e1f5e5438f4.cram.crai46
1a5e47cb-6b89-44d4-821e-6e1f5e5438f4.g.vcf.gz45
1a5e47cb-6b89-44d4-821e-6e1f5e5438f4.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_K6D0448D
associated files:1a5e47cb-6b89-44d4-821e-6e1f5e5

Could not find  for BS_1R42YJ59
associated files:2021b989-c13b-4862-9ca5-cf327c403e9d.cram41
bf730b8d28364765b749d0d15dd1d7c6.bam36
2021b989-c13b-4862-9ca5-cf327c403e9d.cram.crai46
2021b989-c13b-4862-9ca5-cf327c403e9d.g.vcf.gz45
2021b989-c13b-4862-9ca5-cf327c403e9d.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_1R42YJ59
associated files:2021b989-c13b-4862-9ca5-cf327c403e9d.cram41
bf730b8d28364765b749d0d15dd1d7c6.bam36
2021b989-c13b-4862-9ca5-cf327c403e9d.cram.crai46
2021b989-c13b-4862-9ca5-cf327c403e9d.g.vcf.gz45
2021b989-c13b-4862-9ca5-cf327c403e9d.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_XSM37R34
associated files:22d53b06-e72f-4543-8b89-67df4fa3f248.cram41
c4edb8ee898d4d0b80434852e7ff3b51.bam36
22d53b06-e72f-4543-8b89-67df4fa3f248.cram.crai46
22d53b06-e72f-4543-8b89-67df4fa3f248.g.vcf.gz45
22d53b06-e72f-454

Could not find  for BS_YEVVAH64
associated files:28b90b2e-4d1c-4322-af47-6d626375e50b.cram41
96ab1277458b45739b8787781085fbb3.bam36
28b90b2e-4d1c-4322-af47-6d626375e50b.cram.crai46
28b90b2e-4d1c-4322-af47-6d626375e50b.g.vcf.gz45
28b90b2e-4d1c-4322-af47-6d626375e50b.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_YEVVAH64
associated files:28b90b2e-4d1c-4322-af47-6d626375e50b.cram41
96ab1277458b45739b8787781085fbb3.bam36
28b90b2e-4d1c-4322-af47-6d626375e50b.cram.crai46
28b90b2e-4d1c-4322-af47-6d626375e50b.g.vcf.gz45
28b90b2e-4d1c-4322-af47-6d626375e50b.g.vcf.gz.tbi49
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz43
9f157377-ee64-4608-bc63-d66772a64e31.vcf.gz.tbi47
Could not find  for BS_RRAYMWSG
associated files:273caca9-a8f2-4274-86d3-d1c13e4a783b.cram41
140238dd0c2e4c6a8fea7fafd4db2840.bam36
273caca9-a8f2-4274-86d3-d1c13e4a783b.cram.crai46
273caca9-a8f2-4274-86d3-d1c13e4a783b.g.vcf.gz45
273caca9-a8f2-427

...

Could not find  for BS_2EJSYSEX
associated files:99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.g.vcf.gz45
14739aa4ae0547ef8917ee6d4b0cdec8.bam36
99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.g.vcf.gz.tbi49
99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.cram41
99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.cram.crai46
Could not find  for BS_2EJSYSEX
associated files:99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.g.vcf.gz45
14739aa4ae0547ef8917ee6d4b0cdec8.bam36
99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.g.vcf.gz.tbi49
99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.cram41
99c0c931-eba6-4e8f-b4b7-fdb9a310e5b8.cram.crai46
Could not find  for BS_SXK1DM65
associated files:9b3d066a-4811-4d52-9d00-10d48ff171d7.g.vcf.gz45
bbcf4645811b44bb94caaabda14d694b.bam36
9b3d066a-4811-4d52-9d00-10d48ff171d7.g.vcf.gz.tbi49
9b3d066a-4811-4d52-9d00-10d48ff171d7.cram41
9b3d066a-4811-4d52-9d00-10d48ff171d7.cram.crai46
Could not find  for BS_SXK1DM65
associated files:9b3d066a-4811-4d52-9d00-10d48ff171d7.g.vcf.gz45
bbcf4645811b44bb94caaabda14d694b.bam36
9b3d066a-4811-4d52-9d

Could not find  for BS_7NCKKZ9V
associated files:bca18f0d-110c-4404-b07f-007a8f970ef3.g.vcf.gz45
1daf5a95401e4be8a014dc5cfb8b8556.bam36
bca18f0d-110c-4404-b07f-007a8f970ef3.g.vcf.gz.tbi49
bca18f0d-110c-4404-b07f-007a8f970ef3.cram41
bca18f0d-110c-4404-b07f-007a8f970ef3.cram.crai46
Could not find  for BS_7NCKKZ9V
associated files:bca18f0d-110c-4404-b07f-007a8f970ef3.g.vcf.gz45
1daf5a95401e4be8a014dc5cfb8b8556.bam36
bca18f0d-110c-4404-b07f-007a8f970ef3.g.vcf.gz.tbi49
bca18f0d-110c-4404-b07f-007a8f970ef3.cram41
bca18f0d-110c-4404-b07f-007a8f970ef3.cram.crai46
Could not find  for BS_AE02QWKC
associated files:bc5ff427-0b60-4eac-8611-916b30098072.g.vcf.gz45
c9bf08b1bf7447a3b8b5d94a7ebd0580.bam36
bc5ff427-0b60-4eac-8611-916b30098072.g.vcf.gz.tbi49
bc5ff427-0b60-4eac-8611-916b30098072.cram41
bc5ff427-0b60-4eac-8611-916b30098072.cram.crai46
Could not find  for BS_AE02QWKC
associated files:bc5ff427-0b60-4eac-8611-916b30098072.g.vcf.gz45
c9bf08b1bf7447a3b8b5d94a7ebd0580.bam36
bc5ff427-0b60-4eac-86

Could not find  for BS_HC90EM2Q
associated files:ce4befc9-8d1b-4928-a2b6-f84d0e87af04.g.vcf.gz45
a470a639300347aa9cbf4b1a26204541.bam36
ce4befc9-8d1b-4928-a2b6-f84d0e87af04.g.vcf.gz.tbi49
ce4befc9-8d1b-4928-a2b6-f84d0e87af04.cram41
ce4befc9-8d1b-4928-a2b6-f84d0e87af04.cram.crai46
Could not find  for BS_HC90EM2Q
associated files:ce4befc9-8d1b-4928-a2b6-f84d0e87af04.g.vcf.gz45
a470a639300347aa9cbf4b1a26204541.bam36
ce4befc9-8d1b-4928-a2b6-f84d0e87af04.g.vcf.gz.tbi49
ce4befc9-8d1b-4928-a2b6-f84d0e87af04.cram41
ce4befc9-8d1b-4928-a2b6-f84d0e87af04.cram.crai46
Could not find  for BS_1DH5E90H
associated files:d19cfcb1-c555-4d16-b7fb-f7554a06b892.g.vcf.gz45
808e3d7455d945f183af6c9a9819f8c1.bam36
d19cfcb1-c555-4d16-b7fb-f7554a06b892.g.vcf.gz.tbi49
d19cfcb1-c555-4d16-b7fb-f7554a06b892.cram41
d19cfcb1-c555-4d16-b7fb-f7554a06b892.cram.crai46
Could not find  for BS_1DH5E90H
associated files:d19cfcb1-c555-4d16-b7fb-f7554a06b892.g.vcf.gz45
808e3d7455d945f183af6c9a9819f8c1.bam36
d19cfcb1-c555-4d16-b7

Could not find  for BS_Q3NS8HZD
associated files:e1d0e620-b846-4188-b052-a67ef31a340a.g.vcf.gz45
85a8bf917615484b912544c9b83e53cb.bam36
e1d0e620-b846-4188-b052-a67ef31a340a.g.vcf.gz.tbi49
e1d0e620-b846-4188-b052-a67ef31a340a.cram41
e1d0e620-b846-4188-b052-a67ef31a340a.cram.crai46
Could not find  for BS_Q3NS8HZD
associated files:e1d0e620-b846-4188-b052-a67ef31a340a.g.vcf.gz45
85a8bf917615484b912544c9b83e53cb.bam36
e1d0e620-b846-4188-b052-a67ef31a340a.g.vcf.gz.tbi49
e1d0e620-b846-4188-b052-a67ef31a340a.cram41
e1d0e620-b846-4188-b052-a67ef31a340a.cram.crai46
Could not find  for BS_3T2JKTXQ
associated files:e06f1024-70f7-443d-8704-2c1043f06485.g.vcf.gz45
27cebfe412624cc39f5e8ea84652ce3b.bam36
e06f1024-70f7-443d-8704-2c1043f06485.g.vcf.gz.tbi49
e06f1024-70f7-443d-8704-2c1043f06485.cram41
e06f1024-70f7-443d-8704-2c1043f06485.cram.crai46
Could not find  for BS_3T2JKTXQ
associated files:e06f1024-70f7-443d-8704-2c1043f06485.g.vcf.gz45
27cebfe412624cc39f5e8ea84652ce3b.bam36
e06f1024-70f7-443d-87

Could not find  for BS_AF2MC2SY
associated files:efb3acfb-abb2-461f-8326-f8b5f9aa85df.g.vcf.gz45
383d819856594cdc8ce41d0e2dd99650.bam36
efb3acfb-abb2-461f-8326-f8b5f9aa85df.g.vcf.gz.tbi49
efb3acfb-abb2-461f-8326-f8b5f9aa85df.cram41
efb3acfb-abb2-461f-8326-f8b5f9aa85df.cram.crai46
Could not find  for BS_AF2MC2SY
associated files:efb3acfb-abb2-461f-8326-f8b5f9aa85df.g.vcf.gz45
383d819856594cdc8ce41d0e2dd99650.bam36
efb3acfb-abb2-461f-8326-f8b5f9aa85df.g.vcf.gz.tbi49
efb3acfb-abb2-461f-8326-f8b5f9aa85df.cram41
efb3acfb-abb2-461f-8326-f8b5f9aa85df.cram.crai46
Processed 2350 bs ids
Could not find  for BS_DE1HY8QE
associated files:eb9e32b6-c2d9-4222-b03b-a09c245ec834.g.vcf.gz45
36e2a0d242004119b53748a13a328d19.bam36
eb9e32b6-c2d9-4222-b03b-a09c245ec834.g.vcf.gz.tbi49
eb9e32b6-c2d9-4222-b03b-a09c245ec834.cram41
eb9e32b6-c2d9-4222-b03b-a09c245ec834.cram.crai46
Could not find  for BS_DE1HY8QE
associated files:eb9e32b6-c2d9-4222-b03b-a09c245ec834.g.vcf.gz45
36e2a0d242004119b53748a13a328d19.bam36